WORK IN PROGRESS!! :-O

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from IPython.display import HTML

# Competition goal

* Classifiy the etiology (origin) of a blood clot shown in a tissue slice of a patient that has experienced an acute ischemic stroke. 
* It's a **binary classification task** with labels CE (cardioembolic) or LAA (Large Artery Atherosclerosis)
* There are supplemental slides with a either an **unknown etiology or an etiology other than CE or LAA**! 

Aha! ... :-O Do you know the difference between CE and LAA or what is meant by an acute ischemic stroke? ... Neither do I! ;-) 

In [ ]:
HTML('<iframe width="800" height="444" src="https://www.youtube.com/embed/abxcrAvw-O0" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

## Table of contents

1. [Prepare to start](#prepare)
2. [Exploratory analysis - EDA](#eda)
3. [The nightmare of overfitting](#nightmare)
4. [Tell me what do you see?](#explain)
    * [SHAP - explain it with GradiendExplainer](#shap)


# Prepare to start <a class="anchor" id="prepare"></a>

## Loading packages

In [ ]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image

import tensorflow as tf

from tqdm.notebook import tqdm


import seaborn as sns
sns.set()

import shap

## Loading data

In [ ]:
train = pd.read_csv("../input/mayo-clinic-strip-ai/train.csv")

# Exploratory analysis <a class="anchor" id="eda"></a>

## What do we know about our training data?

In [ ]:
train.head()

In [ ]:
train.shape

* The **image_id** consists of two different parts: *{patient_id}_{image_number}*
* The **center_id** defines the medical center or institute where the image was taken
* **patient_id** is the id of the patient
* **image_num** is a counter for slices of a patient
* **label** is our target
    * CE: cardioembolic
    * LAA: Large artery atherosclerosis
    

In [ ]:
train.patient_id.nunique()

632 patients in train.

In [ ]:
plt.figure(figsize=(10,5))
sns.countplot(train.groupby("patient_id").image_num.size(), palette="Greens_r")
plt.xlabel("Number of images per patient")
plt.title("Max image number per patient in train");

We can clearly see that most patients only have one slice in the training data and only a few show more than one. Maybe these are hard cases where it's more difficult to see the etiology?! Or does it depend on the medical center?

In [ ]:
train.groupby("patient_id").center_id.nunique().max()

Ok, all patients belong to one center. There is no patient that shows images that were taken in different medial centers. How many do we have and how many patients do we have per center?

In [ ]:
train.groupby("center_id").patient_id.size().sort_values(ascending=False)

Definitely not balanced.

Let's take a look at the target distribution! :-)

In [ ]:
sns.countplot(train.label, palette="Reds_r")

Ok, it's an imbalanced classification problem. 

In [ ]:
train.label.value_counts() / train.shape[0]

Ok we know that we have some patients with more than one slice. Is it possible they show more than one label per patient?

In [ ]:
train.groupby("patient_id").label.nunique().max()

No! :-)

In [ ]:
test = pd.read_csv("../input/mayo-clinic-strip-ai/test.csv")
test["image_path"] = test["image_id"].apply(lambda x: os.path.join("../input/mayo-clinic-strip-ai/test/", x+".tif"))
test.head()

## The nightmare of overfitting ;-) <a class="anchor" id="nightmare"></a>

Can't wait anymore! Let's start to discover the images! 

In [ ]:
from PIL import Image

Image.MAX_IMAGE_PIXELS = None

In [ ]:
example = "../input/mayo-clinic-strip-ai/train/006388_0.tif"
img = Image.open(example)
img.size

Uff!

In [ ]:
x1 = img.size[0]
x2 = img.size[1]
sc = x2/x1

In [ ]:
factor = 1/10
resized_img = img.resize((int(x1*factor), int(sc*x1*factor)))

In [ ]:
resized_img.size

In [ ]:
resized_img.rotate(90, expand=True)

Oh this will be a nightmare for validation.... such a complex feature space... I can't see the blood clot, do you? How should we prevent our model from overfitting? Seems hopeless. Now, imagine that medical centers might also use different methods or protocols to obtain these slices... ohoh! The number of patients given such a high feature space is low and the public test data seems to be quite small as well. Hmm.

# Tell me, what do you see? <a class="anchor" id="explain"></a>

Ok, so far I can't imagine that any model will be able to see something useful in this complex feature space... but let's better try to understand and explain what our models "see". For this purpose I like to use and learn more about some machine learning explainability tools for computer vision.

# Building a model



# SHAP - Explain it with GradientExplainer <a class="anchor" id="shap"></a>

* [Example for Gradient Explainer using Keras](https://github.com/slundberg/shap/blob/master/notebooks/image_examples/image_classification/Explain%20an%20Intermediate%20Layer%20of%20VGG16%20on%20ImageNet.ipynb)

In [ ]:
def map2layer(x, layer):
    feed_dict = dict(zip([model.layers[0].input], [preprocess_input(x.copy())]))
    return K.get_session().run(model.get_layer("efficientnetb6").get_layer(layer).input, feed_dict)

In [ ]:
#e = shap.GradientExplainer((model.get_layer("efficientnetb6").get_layer("top_conv").input,
                            #model.get_layer("efficientnetb6").get_layer("top_conv").output), 
                            #map2layer(preprocess_input(X.copy()), "top_conv"))